In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from visualization_utilities import *
from metadata import *
from utilities2015 import *

Setting environment for Local Macbook Pro


In [2]:
from pandas import *

# New version

In [12]:
contours_df = DataManager.load_annotation_v4(stack='MD594', by_human=True, timestamp='latest', suffix='contours')

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD594" "/home/yuncong/CSHL_labelings_v3/MD594" --exclude "*" --include "*contours*"
latest timestamp:  07302017183604


In [13]:
contours_df[contours_df['type'] == 'confirmed']

,class,creator,downsample,edits,id,label_position,name,orientation,parent_structure,section,side,side_manually_assigned,time_created,type,vertices,filename
110033426909007,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'073...",110033426909007,"[29646.9977325, 7213.06376988]",Cb,sagittal,[],210,None,False,07302017172153,confirmed,"[[27502.781909, 8930.67922729], [27187.6989547...",MD594-IHC44-2015.08.26-17.54.29_MD594_2_0131
113316500167164,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'072...",113316500167164,"[21695.3663444, 7293.67359985]",Cb,sagittal,[],100,None,False,07292017055254,confirmed,"[[19701.1334131, 9876.35283144], [19467.192683...",MD594-IHC17-2015.08.26-16.15.19_MD594_2_0050
126089258811500,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'073...",126089258811500,"[28358.9333066, 6533.24113771]",Cb,sagittal,[],242,None,False,07302017181209,confirmed,"[[25704.3778787, 7821.75966327], [25564.054804...",MD594-IHC53-2015.08.26-18.29.03_MD594_1_0157
127975655130773,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'073...",127975655130773,"[23969.5395456, 6579.06737952]",Cb,sagittal,[],304,None,False,07302017181724,confirmed,"[[21644.655959, 8654.4768638], [21416.1347923,...",MD594-IHC67-2015.08.26-19.23.30_MD594_3_0201
130658524255867,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'072...",130658524255867,"[27567.0659808, 6125.16944923]",Cb,sagittal,[],140,None,False,07292017055529,confirmed,"[[25169.7730391, 7841.00697164], [25009.916366...",MD594-IHC27-2015.08.26-16.50.00_MD594_1_0079
143275320340084,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'071...",143275320340084,"[23792.6366169, 8017.87726348]",DC,sagittal,[],305,R,True,07192017071039,confirmed,"[[23724.6793674, 8043.41407086], [23643.029126...",MD594-N68-2015.08.27-01.01.13_MD594_1_0202
171248160288798,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'073...",171248160288798,"[22087.2384951, 6419.13137434]",Cb,sagittal,[],333,None,False,07302017182002,confirmed,"[[19404.2631671, 8374.88555449], [19326.394083...",MD594-N72-2015.08.27-01.18.48_MD594_3_0216
176142125636993,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'073...",176142125636993,"[28803.2994913, 6156.84799696]",Cb,sagittal,[],190,None,False,07302017153704,confirmed,"[[26383.2721813, 7700.48363701], [26194.632272...",MD594-IHC39-2015.08.26-17.34.24_MD594_2_0116
176390099822253,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'071...",176390099822253,"[29035.1171636, 8339.72431404]",DC,sagittal,[],155,L,True,07192017070344,confirmed,"[[28998.7331752, 8382.09639195], [28961.540584...",MD594-N31-2015.08.26-22.36.37_MD594_1_0091
185873108061303,contour,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'071...",185873108061303,"[28376.7010816, 8908.98885254]",DC,sagittal,[],150,L,True,07192017070213,confirmed,"[[28321.8893696, 8939.67225787], [28268.555372...",MD594-IHC29-2015.08.26-16.56.59_MD594_2_0086


# Old version

In [3]:
stack = 'MD589'

In [4]:
contours = read_hdf('/home/yuncong/CSHL_labelings_v3/%(stack)s/%(stack)s_annotation_v3.h5' % {'stack': stack})

In [5]:
contours

,creator,downsample,edits,flags,id,label_position,name,orientation,parent_structure,section,side,side_manually_assigned,time_created,vertices,filename
100178860389859,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",1,100178860389859,"[17247.0823677, 10919.7426844]",RMC,sagittal,[],255,R,False,09222016035339,"[[16988.9491423, 11411.0524856], [16861.232581...",MD585-N45-2015.07.16-22.40.26_MD585_2_0134
100191080909744,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",0,100191080909744,"[26931.8170015, 11267.7025549]",12N,sagittal,[],219,S,False,09222016025459,"[[26747.2196985, 11322.661802], [26723.2035819...",MD585-N39-2015.07.16-22.09.19_MD585_2_0116
100242155386645,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",1,100242155386645,"[22048.7016219, 10766.3339918]",VLL,sagittal,[],301,R,False,09222016042430,"[[22054.3869495, 11354.3162472], [21894.405983...",MD585-N53-2015.08.19-16.16.47_MD585_1_0157
100288825583501,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",0,100288825583501,"[26277.6864602, 12085.1357065]",SolIM,sagittal,[],175,None,False,09222016194614,"[[25958.6864191, 12146.5712428], [25659.055469...",MD585-N32-2015.08.19-16.07.10_MD585_1_0094
100311401158873,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",1,100311401158873,"[19800.378291, 11618.759015]",8n,sagittal,[],125,L,False,09222016153350,"[[19816.8058802, 11913.3906178], [19657.363229...",MD585-N23-2015.07.16-20.51.24_MD585_3_0069
100326455666302,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",0,100326455666302,"[23559.0841215, 6702.46174242]",IC,sagittal,[],140,S,False,09222016013635,"[[23961.3958333, 5444.53571667], [24227.580512...",MD585-IHC26-2015.08.19-13.23.15_MD585_1_0076
100440544644031,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",1,100440544644031,"[18769.6913919, 11631.2338112]",VLL,sagittal,[],286,R,False,09222016042430,"[[19260.2255195, 12548.7665639], [19108.345730...",MD585-IHC50-2015.07.16-18.02.54_MD585_2_0149
100671880710742,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",1,100671880710742,"[19167.438079, 5450.61492507]",SC,sagittal,[],239,S,False,09222016044114,"[[17617.9916109, 6917.10760853], [17458.067530...",MD585-N42-2015.07.16-22.24.42_MD585_3_0126
100718174191046,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",1,100718174191046,"[23692.1254176, 14224.2421154]",7N,sagittal,[],173,L,False,09222016032121,"[[24019.471166, 14682.6337449], [23859.4869399...",MD585-N31-2015.08.19-16.01.47_MD585_3_0093
100920357531201,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'092...",0,100920357531201,"[20414.6691112, 13073.077174]",Tz,sagittal,[],240,R,False,09222016030445,"[[19855.1742478, 13068.1143797], [19732.686403...",MD585-IHC42-2015.08.19-14.26.30_MD585_3_0126


In [7]:
from annotation_utilities import *

In [8]:
contours = contours[(contours['orientation'] == 'sagittal') & (contours['downsample'] == 1)]
contours = convert_annotation_v3_original_to_aligned_cropped(contours, stack=stack)

In [10]:
grouped = contours.groupby('section')

In [15]:
for sec, group in grouped:
    for contour_id, contour in group.iterrows():
        print contour_id
    print '\n'

24041485827291


178638821776884


225468065711699


82753828715844


108639196063443
240450955921646


139995532033938
276274803125745


123201065869455
50599366457376


128891739813762
280008487571593
74734208874016


137576545852286
139884971202912
44979814970993


198648392198851
252434421302035
53849871852705


215403441631652
272974593078035
32236200064177


165920538830976
243256546173591
58906621711441


175994136617351
258516032496448
45531577507434


187863059163195
193809479990018
245020993386827


124244720326548
28990358901640
6039163938366


13783358246531
269642137367206
93951936437221


170927976283983
250086639456848
65686469868581


12994163947428
131268714517286
180619860209674
3348124243740


140881450963675
171250512966690
277597751075951
90654685894305


124892661563969
202987135024252
207964402507211
50422758984114
63049897456070


236059505041962
259084303736308
69129898116566
86007295995378
95926547893398


111731759580942
135434889301284
265153680649971
273505